In [ ]:
import requests
import os
import time
import base64
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()

# Configuration
API_BASE_URL = "https://marble2-kgw-prod-iac1.wlt-ai.art/api/v1"
BEARER_TOKEN = os.getenv("MARBLE_API_TOKEN")  # Set this in your environment

if not BEARER_TOKEN:
    raise ValueError("MARBLE_API_TOKEN environment variable is not set")

headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "Content-Type": "application/json"
}

In [12]:
def encode_image_to_base64(image_path):
    """Read an image file and encode it to base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def create_world(image_path=None, text_prompt=None, model="Marble 0.1-mini", is_pano=False):
    """
    Create a new world generation request.
    
    Args:
        image_path: Path to image file
        text_prompt: Text description
        model: Model name (default: "Marble 0.1-mini")
        is_pano: Whether the input is a panorama (default: False)
    
    Returns:
        dict: Response with world ID and initial status
    """

    # Get image data
    image_base64 = encode_image_to_base64(image_path)
    extension = Path(image_path).suffix[1:]  # Remove the dot
    
    # Construct request body
    body = {
        "id": "",
        "display_name": None,
        "status": "PENDING",
        "owner_id": None,
        "created_at": 0,
        "updated_at": 0,
        "generation_input": {
            "seed": None,
            "model": model,
            "prompt": {
                "text_prompt": text_prompt,
                "image_prompt": {
                    "data_base64": image_base64,
                    "extension": extension
                } if image_base64 else None,
                "is_pano": is_pano
            }
        },
        "permission": {
            "public": True
        }
    }
    
    # Make POST request
    response = requests.post(
        f"{API_BASE_URL}/worlds",
        headers=headers,
        json=body
    )
    
    response.raise_for_status()
    return response.json()


In [13]:
def get_world_status(world_id):
    """
    Get the current status of a world generation.
    
    Args:
        world_id: The world ID returned from create_world
    
    Returns:
        dict: Current world status and data
    """
    headers = {
        "Authorization": f"Bearer {BEARER_TOKEN}"
    }
    
    response = requests.get(
        f"{API_BASE_URL}/worlds/{world_id}",
        headers=headers
    )
    
    response.raise_for_status()
    return response.json()


In [ ]:
def wait_for_completion(world_id, poll_interval=5, max_wait_time=600):
    """
    Poll the API until the world generation is complete.
    
    Args:
        world_id: The world ID to poll
        poll_interval: Seconds between polls (default: 5)
        max_wait_time: Maximum time to wait in seconds (default: 600)
    
    Returns:
        dict: Final world data with generation_output
    """
    start_time = time.time()
    
    while True:
        # Check if we've exceeded max wait time
        if time.time() - start_time > max_wait_time:
            raise TimeoutError(f"World generation exceeded maximum wait time of {max_wait_time}s")
        
        # Get current status
        world_data = get_world_status(world_id)
        status = world_data["status"]
        
        print(f"Status: {status} (elapsed: {int(time.time() - start_time)}s)")
        
        if status == "SUCCEEDED":
            print("✓ Generation completed successfully!")
            return world_data
        elif status == "FAILED":
            error = world_data.get("error", "Unknown error")
            raise RuntimeError(f"World generation failed: {error}")
        elif status in ["PENDING", "INITIALIZING", "PROCESSING"]:
            # Still processing, wait and poll again
            time.sleep(poll_interval)
        else:
            print(f"Warning: Unknown status '{status}', continuing to poll...")
            time.sleep(poll_interval)


In [19]:
# Example: Create world from image
image_path = "/Users/mingjun/prompt.jpg"
text_prompt = "The scene is a tranquil village courtyard, captured in a realistic style, evoking a sense of ancient charm and quiet history. The overall tone is peaceful and timeless, with warm sunlight illuminating weathered stone structures and lush greenery. The courtyard is paved with irregularly shaped cobblestones, radiating outwards from a central stone fountain that features a bowl-shaped basin filled with water. Stone buildings, multi-storied and adorned with climbing vines, encircle the courtyard, creating a secluded atmosphere. Arched doorways and windows with light blue wooden shutters are set into the textured stone walls, suggesting residences or small shops. A prominent stone staircase ascends along one of the buildings, leading to upper levels, with potted plants placed on its steps and landings. Various potted plants, including small trees and flowering shrubs, are strategically positioned around the courtyard, adding touches of vibrant green against the earthy tones of the stone. A wooden bench or table is tucked into an alcove on the left, offering a place for rest. The stone structures appear well-maintained, with a patina that speaks to centuries of existence. The arrangement of the buildings creates winding pathways and shaded nooks within the courtyard. The fountain stands as the undisputed focal point, with cobblestones forming a circular pattern around its base. The staircase extends upwards from the right side of the courtyard, curving gently as it ascends. The numerous potted plants are scattered throughout the space, enhancing the natural feel of the stone environment."

initial_response = create_world(
    image_path=image_path,
    text_prompt=text_prompt
)

world_id = initial_response["id"]
print(f"World ID: {world_id}")
print(f"Initial status: {initial_response['status']}")


World ID: 2d48efb1-b721-409a-8ac7-a35c4c825372
Initial status: INITIALIZING


In [20]:
# Wait for completion and get results
final_data = wait_for_completion(world_id)

print("\n=== Generation Output ===")
generation_output = final_data.get("generation_output", {})

print(f"Display Name: {final_data.get('display_name')}")
print(f"\nAvailable outputs:")
print(f"  PLY URL: {generation_output.get('ply_url')}")
print(f"  WLG URL: {generation_output.get('wlg_url')}")
print(f"  Collider Mesh: {generation_output.get('collider_mesh_url')}")
print(f"  SPZ URLs: {generation_output.get('spz_urls')}")


{'id': '2d48efb1-b721-409a-8ac7-a35c4c825372', 'display_name': None, 'status': 'INITIALIZING', 'error': None, 'owner_id': 'grlI59jyIRRiLmku73bea68qRn52', 'created_at': 1759567488, 'updated_at': 1759567488, 'permission': {'public': True, 'allowed_readers': [], 'allowed_writers': []}, 'stats': None, 'application_data': {'owner_username': 'mongj', 'owner_display_name': 'Ming Jun Zhang', 'reactions': None, 'pinned': False}, 'generation_input': {'model': 'Marble 0.1-mini', 'seed': None, 'prompt': {'image_prompt': {'uri': 'https://cdn.marble.worldlabs.ai/2d48efb1-b721-409a-8ac7-a35c4c825372/9d8e3da7_image_prompt.jpg', 'data_base64': None, 'extension': None}, 'text_prompt': 'The scene is a tranquil village courtyard, captured in a realistic style, evoking a sense of ancient charm and quiet history. The overall tone is peaceful and timeless, with warm sunlight illuminating weathered stone structures and lush greenery. The courtyard is paved with irregularly shaped cobblestones, radiating outwa